In [1]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4877462453921777305]

In [6]:
from __future__ import (absolute_import, 
                        division, 
                        print_function, 
                        unicode_literals)

import os
import glob
import time
import math
import random
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import pylab
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import tensorflow as tf
from PIL import Image
from mpl_toolkits.axes_grid1 import ImageGrid
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten

# Import models and config
from models import vanilla,vgg16

# Set random seed for consistency
random.seed(100)
np.random.seed(100)
tf.random.set_seed(100)

# Displays all outputs of a cell (not just last output)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 5]

print('TensorFlow version:', tf.__version__)
print('Keras version:', tf.keras.__version__)

img_height = 224
img_width = 224
num_channels = 3
input_shape = (img_height, img_width, num_channels)
num_classes = 4

batch_size = 32
epochs = 30
learning_rate = 2e-5


TensorFlow version: 2.1.0
Keras version: 2.2.4-tf


In [5]:

work_dir = '.'
data_dir = os.path.join(work_dir, 'data')

# Original data (before split)
black_measles_dir = os.path.join(data_dir, 'black_measles')
black_rot_dir = os.path.join(data_dir, 'black_rot')
leaf_blight_dir = os.path.join(data_dir, 'leaf_blight')
healthy_dir = os.path.join(data_dir, 'healthy')

# Train/Validation/Test
train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'validation')
test_dir = os.path.join(data_dir, 'test')

# Training data by class
train_black_measles_dir = os.path.join(train_dir, 'black_measles')
train_black_rot_dir = os.path.join(train_dir, 'black_rot')
train_leaf_blight_dir = os.path.join(train_dir, 'leaf_blight')
train_healthy_dir = os.path.join(train_dir, 'healthy')

# Validation data by class
validation_black_measles_dir = os.path.join(validation_dir, 'black_measles')
validation_black_rot_dir = os.path.join(validation_dir, 'black_rot')
validation_leaf_blight_dir = os.path.join(validation_dir, 'leaf_blight')
validation_healthy_dir = os.path.join(validation_dir, 'healthy')

# Test data by class
test_black_measles_dir = os.path.join(test_dir, 'black_measles')
test_black_rot_dir = os.path.join(test_dir, 'black_rot')
test_leaf_blight_dir = os.path.join(test_dir, 'leaf_blight')
test_healthy_dir = os.path.join(test_dir, 'healthy')

In [6]:
num_black_measles = len(os.listdir(black_measles_dir))
num_black_rot = len(os.listdir(black_rot_dir))
num_leaf_blight = len(os.listdir(leaf_blight_dir))
num_healthy = len(os.listdir(healthy_dir))
total_images = num_black_measles + num_black_rot + num_leaf_blight + num_healthy
                                   
print('Total Images: {}'.format(total_images))
print('black_measles class has {} images'.format(num_black_measles))
print('black_rot class has {} images'.format(num_black_rot))
print('leaf_blight class has {} images'.format(num_leaf_blight))
print('healthy class has {} images'.format(num_healthy))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '.\\data\\black_measles'

In [7]:

# Sanity check 
num_train_black_measles = len(os.listdir(train_black_measles_dir))
num_train_black_rot = len(os.listdir(train_black_rot_dir))
num_train_leaf_blight = len(os.listdir(train_leaf_blight_dir))
num_train_healthy = len(os.listdir(train_healthy_dir))

num_val_black_measles = len(os.listdir(validation_black_measles_dir))
num_val_black_rot = len(os.listdir(validation_black_rot_dir))
num_val_leaf_blight = len(os.listdir(validation_leaf_blight_dir))
num_val_healthy = len(os.listdir(validation_healthy_dir))

num_test_black_measles = len(os.listdir(test_black_measles_dir))
num_test_black_rot = len(os.listdir(test_black_rot_dir))
num_test_leaf_blight = len(os.listdir(test_leaf_blight_dir))
num_test_healthy = len(os.listdir(test_healthy_dir))

total_train = num_train_black_measles + num_train_black_rot + \
                num_train_leaf_blight + num_train_healthy
total_val = num_val_black_measles + num_val_black_rot + \
              num_val_leaf_blight + num_val_healthy
total_test = num_test_black_measles + num_test_black_rot + \
              num_test_leaf_blight + num_test_healthy

def check_distribution(total, black_measles, black_rot, leaf_blight, healthy):

    print('% of black_measles: {:.2f}%'.format((black_measles/total)*100.0))
    print('% of blacK_rot: {:.2f}%'.format((black_rot / total)*100))
    print('% of leaf_blight: {:.2f}%'.format((leaf_blight / total)*100.0))
    print('% of healthy: {:.2f}%'.format((healthy / total)*100.0))
    print()
    
print('Total images:', total_train + total_val + total_test)
print('Total black_measles:', num_train_black_measles+num_val_black_measles+num_test_black_measles)
print('Total blacK_rot:', num_train_black_rot+num_val_black_rot+num_test_black_rot)
print('Total leaf_blight:', num_train_leaf_blight+num_val_leaf_blight+num_test_leaf_blight)
print('Total healthy:', num_train_healthy+num_val_healthy+num_test_healthy)
print()
print('Number of black_measles training images:', num_train_black_measles)
print('Number of black_rot training images:', num_train_black_rot)
print('Number of leaf_blight training images:', num_train_leaf_blight)
print('Number of healthy training images:', num_train_healthy)
print()
print('Number of black_measles validation images:', num_val_black_measles)
print('Number of black_rot validation images:', num_val_black_rot)
print('Number of leaf_blight validation images:', num_val_leaf_blight)
print('Number of healthy validation images:', num_val_healthy)
print()
print('Number of black_measles test images:', num_test_black_measles)
print('Number of black_rot test images:', num_test_black_rot)
print('Number of leaf_blight test images:', num_test_leaf_blight)
print('Number of healthy test images:', num_test_healthy)
print()
print('Training distribution')
check_distribution(total_train, num_train_black_measles, num_train_black_rot,
                    num_train_leaf_blight, num_train_healthy)
print('Validation distribution')
check_distribution(total_val, num_val_black_measles, num_val_black_rot,
                    num_val_leaf_blight, num_val_healthy)
print('Test distribution')
check_distribution(total_test, num_test_black_measles, num_test_black_rot,
                    num_test_leaf_blight, num_test_healthy)

Total images: 4062
Total black_measles: 1383
Total blacK_rot: 1180
Total leaf_blight: 1076
Total healthy: 423

Number of black_measles training images: 886
Number of black_rot training images: 756
Number of leaf_blight training images: 689
Number of healthy training images: 272

Number of black_measles validation images: 221
Number of black_rot validation images: 188
Number of leaf_blight validation images: 172
Number of healthy validation images: 67

Number of black_measles test images: 276
Number of black_rot test images: 236
Number of leaf_blight test images: 215
Number of healthy test images: 84

Training distribution
% of black_measles: 34.04%
% of blacK_rot: 29.04%
% of leaf_blight: 26.47%
% of healthy: 10.45%

Validation distribution
% of black_measles: 34.10%
% of blacK_rot: 29.01%
% of leaf_blight: 26.54%
% of healthy: 10.34%

Test distribution
% of black_measles: 34.03%
% of blacK_rot: 29.10%
% of leaf_blight: 26.51%
% of healthy: 10.36%



In [8]:

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    target_size=(img_height, img_width),
                    color_mode='rgb',
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True, 
                    seed=42)

validation_generator = test_datagen.flow_from_directory(
                          validation_dir,
                          target_size=(img_height, img_width),
                          color_mode='rgb',
                          batch_size=batch_size,
                          class_mode='categorical')

valtest_generator = test_datagen.flow_from_directory(
                          validation_dir,
                          target_size=(img_height, img_width),
                          color_mode='rgb',
                          batch_size=1,
                          class_mode='categorical')

# Test data generator - no augmentation (to be used at very end)
test_gen = test_datagen.flow_from_directory(
                test_dir,
                target_size=(img_height, img_width),
                color_mode='rgb',
                batch_size=1,
                class_mode='categorical',
                shuffle=False)

Found 2603 images belonging to 4 classes.
Found 648 images belonging to 4 classes.
Found 648 images belonging to 4 classes.
Found 811 images belonging to 4 classes.


In [9]:

data = {'train': train_generator,
        'num_train': total_train,
        'val': validation_generator,
        'num_val': total_val,
        'test': test_gen,
        'num_test': total_test,
        'valtest': test_gen}

In [10]:
#trained models
if not os.path.exists('./trained_models'):
    os.makedirs('./trained_models')

In [28]:
model = vanilla.VanillaCNN()
model.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 26, 26, 64)        0   

In [29]:
# Train plain CNN model
model.train(data)



Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 81 steps, validate for 20 steps
Epoch 1/30
81/81 [==============================] - 170s 2s/step - loss: 1.3036 - acc: 0.3477 - val_loss: 1.2140 - val_acc: 0.4406
Epoch 2/30
81/81 [==============================] - 139s 2s/step - loss: 1.1378 - acc: 0.5189 - val_loss: 1.0124 - val_acc: 0.5797
Epoch 3/30
81/81 [==============================] - 114s 1s/step - loss: 0.9729 - acc: 0.5760 - val_loss: 0.8743 - val_acc: 0.6219
Epoch 4/30
81/81 [==============================] - 122s 2s/step - loss: 0.8804 - acc: 0.6188 - val_loss: 0.7908 - val_acc: 0.6344
Epoch 5/30
81/81 [==============================] - 125s 2s/step - loss: 0.7982 - acc: 0.6593 - val_loss: 0.7212 - val_acc: 0.6859
Epoch 6/30
81/81 [==============================] - 127s 2s/step - loss: 0.7222 - acc: 0.7067 - val_loss: 0.6398 - val_acc: 0.7437
Epoch 7/30
81/81 [============================

NotFoundError: Failed to create a directory: ./trained_models\./trained_models/vanillaCNN_model_epochs30\variables; No such file or directory

In [2]:
model='./trained_models/vanilla.h5'
model.load(model)

print(smodel)

AttributeError: 'str' object has no attribute 'load'

In [1]:
# Save model
name = 'vanillaCNN_model_epochs30'
model.save('vanilla.h5')

NameError: name 'model' is not defined

In [1]:

model.evaluate(model, data['test'], data['num_test'], name=None)

NameError: name 'model' is not defined

In [3]:
model = vgg16.VGG16()
model.summary()

58892288/58889256 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [11]:
model.train(data)

# Save model
name = 'vgg16_model_epochs30'
model.save('vgg16.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 81 steps, validate for 20 steps
Epoch 1/30
81/81 [==============================] - 1118s 14s/step - loss: 0.8322 - acc: 0.6604 - val_loss: 0.4046 - val_acc: 0.8828
Epoch 2/30
81/81 [==============================] - 1148s 14s/step - loss: 0.3657 - acc: 0.8775 - val_loss: 0.2740 - val_acc: 0.9250
Epoch 3/30
81/81 [==============================] - 1061s 13s/step - loss: 0.2651 - acc: 0.9152 - val_loss: 0.2234 - val_acc: 0.9422
Epoch 4/30
81/81 [==============================] - 1608s 20s/step - loss: 0.2119 - acc: 0.9319 - val_loss: 0.1881 - val_acc: 0.9484
Epoch 5/30
81/81 [==============================] - 1021s 13s/step - loss: 0.1816 - acc: 0.9459 - val_loss: 0.1683 - val_acc: 0.9469
Epoch 6/30
81/81 [==============================] - 1115s 14s/step - loss: 0.1555 - acc: 0.9545 - val_loss: 0.1517 - val_acc: 0.9547
Epoch 7/30
81/81 [================

'./trained_models\\vgg16.h5'

In [13]:
s="Epoch 1/30 81/81 [==============================] - 1118s 14s/step - loss: 0.8322 - acc: 0.6604 - val_loss: 0.4046 - val_acc: 0.8828
Epoch 2/30
81/81 [==============================] - 1148s 14s/step - loss: 0.3657 - acc: 0.8775 - val_loss: 0.2740 - val_acc: 0.9250
Epoch 3/30
81/81 [==============================] - 1061s 13s/step - loss: 0.2651 - acc: 0.9152 - val_loss: 0.2234 - val_acc: 0.9422
Epoch 4/30
81/81 [==============================] - 1608s 20s/step - loss: 0.2119 - acc: 0.9319 - val_loss: 0.1881 - val_acc: 0.9484
Epoch 5/30
81/81 [==============================] - 1021s 13s/step - loss: 0.1816 - acc: 0.9459 - val_loss: 0.1683 - val_acc: 0.9469
Epoch 6/30
81/81 [==============================] - 1115s 14s/step - loss: 0.1555 - acc: 0.9545 - val_loss: 0.1517 - val_acc: 0.9547
Epoch 7/30
81/81 [==============================] - 1105s 14s/step - loss: 0.1353 - acc: 0.9607 - val_loss: 0.1439 - val_acc: 0.9531
Epoch 8/30
81/81 [==============================] - 1394s 17s/step - loss: 0.1229 - acc: 0.9642 - val_loss: 0.1338 - val_acc: 0.9563
Epoch 9/30
81/81 [==============================] - 1518s 19s/step - loss: 0.1121 - acc: 0.9658 - val_loss: 0.1237 - val_acc: 0.9609
Epoch 10/30
81/81 [==============================] - 1492s 18s/step - loss: 0.1023 - acc: 0.9728 - val_loss: 0.1209 - val_acc: 0.9641
Epoch 11/30
81/81 [==============================] - 1073s 13s/step - loss: 0.0917 - acc: 0.9767 - val_loss: 0.1119 - val_acc: 0.9641
Epoch 12/30
81/81 [==============================] - 1397s 17s/step - loss: 0.0834 - acc: 0.9809 - val_loss: 0.1071 - val_acc: 0.9609
Epoch 13/30
81/81 [==============================] - 1015s 13s/step - loss: 0.0785 - acc: 0.9790 - val_loss: 0.1032 - val_acc: 0.9641
Epoch 14/30
81/81 [==============================] - 1010s 12s/step - loss: 0.0722 - acc: 0.9833 - val_loss: 0.0991 - val_acc: 0.9672
Epoch 15/30
81/81 [==============================] - 1007s 12s/step - loss: 0.0641 - acc: 0.9833 - val_loss: 0.0956 - val_acc: 0.9641
Epoch 16/30
81/81 [==============================] - 1004s 12s/step - loss: 0.0589 - acc: 0.9856 - val_loss: 0.0926 - val_acc: 0.9688
Epoch 17/30
81/81 [==============================] - 1006s 12s/step - loss: 0.0580 - acc: 0.9868 - val_loss: 0.0976 - val_acc: 0.9641
Epoch 18/30
81/81 [==============================] - 1005s 12s/step - loss: 0.0565 - acc: 0.9876 - val_loss: 0.0874 - val_acc: 0.9672
Epoch 19/30
81/81 [==============================] - 1125s 14s/step - loss: 0.0482 - acc: 0.9891 - val_loss: 0.0880 - val_acc: 0.9688
Epoch 20/30
81/81 [==============================] - 1011s 12s/step - loss: 0.0438 - acc: 0.9930 - val_loss: 0.0832 - val_acc: 0.9656
Epoch 21/30
81/81 [==============================] - 1007s 12s/step - loss: 0.0410 - acc: 0.9942 - val_loss: 0.0836 - val_acc: 0.9656
Epoch 22/30
81/81 [==============================] - 1007s 12s/step - loss: 0.0377 - acc: 0.9934 - val_loss: 0.0798 - val_acc: 0.9719
Epoch 23/30
81/81 [==============================] - 1008s 12s/step - loss: 0.0386 - acc: 0.9918 - val_loss: 0.0790 - val_acc: 0.9719
Epoch 24/30
81/81 [==============================] - 1011s 12s/step - loss: 0.0350 - acc: 0.9949 - val_loss: 0.0798 - val_acc: 0.9703
Epoch 25/30
81/81 [==============================] - 1011s 12s/step - loss: 0.0326 - acc: 0.9938 - val_loss: 0.0778 - val_acc: 0.9719
Epoch 26/30
81/81 [==============================] - 1008s 12s/step - loss: 0.0298 - acc: 0.9981 - val_loss: 0.0746 - val_acc: 0.9781
Epoch 27/30
81/81 [==============================] - 1008s 12s/step - loss: 0.0280 - acc: 0.9973 - val_loss: 0.0784 - val_acc: 0.9750
Epoch 28/30
81/81 [==============================] - 1010s 12s/step - loss: 0.0295 - acc: 0.9949 - val_loss: 0.0770 - val_acc: 0.9688
Epoch 29/30
81/81 [==============================] - 1027s 13s/step - loss: 0.0263 - acc: 0.9969 - val_loss: 0.0735 - val_acc: 0.9719
Epoch 30/30
81/81 [==============================] - 1003s 12s/step - loss: 0.0247 - acc: 0.9969 - val_loss: 0.0688 - val_acc: 0.9766"

SyntaxError: EOL while scanning string literal (<ipython-input-13-8d69e98261ec>, line 1)